In [1]:
import module_loader
import pandas as pd
from bookirds.curves import *
from bookirds.dual import Dual

### Define our Market Rates and Solve our Curve

We will create 1y, 2y, 5y and 10y par swap rates

In [2]:
nodes = {
    datetime(2022, 1, 1): Dual(1, {"v0": 1}),
    datetime(2023, 1, 1): Dual(1, {"v1": 1}),
    datetime(2024, 1, 1): Dual(1, {"v2": 1}),
    datetime(2027, 1, 1): Dual(1, {"v3": 1}),
    datetime(2032, 1, 1): Dual(1, {"v4": 1})
}
swaps = {
    Swap(datetime(2022, 1, 1), 12*1, 12, 12): 1.210,
    Swap(datetime(2022, 1, 1), 12*2, 12, 12): 1.635,
    Swap(datetime(2022, 1, 1), 12*5, 12, 12): 1.885,
    Swap(datetime(2022, 1, 1), 12*10, 12, 12): 1.930,
}
s_cv = SolvedCurve(
    nodes=nodes,
    swaps=list(swaps.keys()),
    obj_rates=list(swaps.values()),
    interpolation="log_linear",
    algorithm="levenberg_marquardt"
)
print(s_cv.iterate())

tolerance reached (levenberg_marquardt) after 9 iterations, func: 4.763309217681635e-15


Check that our solved curve is re-pricing our input swap rates

In [3]:
for swap in swaps.keys():
    print(swap.rate(s_cv).real)

1.2100000001274405
1.635000000529199
1.8850000037073773
1.9299999310850648


Yes, close enough!

Lets check that our risk function generates risk only for the exact instrument

In [4]:
risk = {}
for swap in swaps.keys():
    risk.update({swap.end: swap.risk(s_cv)[:, 0]})
    
df = pd.DataFrame(risk, index=["1y", "2y", "5y", "10y"])
df.style.format("{:.3f}")

fwd tolerance reached (gauss_newton) after 2 iterations, func: 1.4636792955140362e-24
bck tolerance reached (gauss_newton) after 2 iterations, func: 2.3284220624110347e-24
fwd tolerance reached (gauss_newton) after 2 iterations, func: 2.028006524066765e-24
bck tolerance reached (gauss_newton) after 2 iterations, func: 1.2489287266656583e-24
fwd tolerance reached (gauss_newton) after 2 iterations, func: 1.0749099552784292e-24
bck tolerance reached (gauss_newton) after 2 iterations, func: 1.0667138876884092e-24
fwd tolerance reached (gauss_newton) after 2 iterations, func: 1.1273410530020714e-24
bck tolerance reached (gauss_newton) after 2 iterations, func: 1.1395583883602621e-24


,2023-01-01 00:00:00,2024-01-01 00:00:00,2027-01-01 00:00:00,2032-01-01 00:00:00
1y,98.804,-0.000,0.000,-0.000
2y,0.000,195.606,0.000,-0.000
5y,0.000,0.000,474.689,-0.000
10y,0.000,0.000,0.000,904.334


That is close enough with tolerance. 

### Forward to Par Jacobian

Now lets consider some forward swaps, and update their rates with our solved curve, and risk these swaps against our curve.

In [5]:
fwd_swaps = {
    Swap(datetime(2022, 1, 1), 12*1, 12, 12): 1,
    Swap(datetime(2023, 1, 1), 12*1, 12, 12): 1,
    Swap(datetime(2024, 1, 1), 12*3, 12, 12): 1,
    Swap(datetime(2027, 1, 1), 12*5, 12, 12): 1,
}
for swap in fwd_swaps.keys():
    rate = swap.rate(s_cv).real
    fwd_swaps[swap] = rate
    swap.set_fixed_rate(fixed_rate=rate)
fwd_swaps

risk = {}
for swap in fwd_swaps.keys():
    risk.update({swap.end: swap.risk(s_cv)[:, 0]})

df = pd.DataFrame(risk, index=["1y", "2y", "5y", "10y"])
df.style.format("{:.3f}")

,2023-01-01 00:00:00,2024-01-01 00:00:00,2027-01-01 00:00:00,2032-01-01 00:00:00
1y,98.804,-99.226,-0.249,-0.045
2y,0.000,196.441,-195.783,-0.181
5y,0.000,0.000,476.315,-474.719
10y,0.000,0.000,0.000,905.642


What we have done above is to risk forward swaps against our curve. When we scale each of these columns to one we have effectively built a Jacobian transformation for forward swaps to par swaps.

In [6]:
J_fwd_par = (df / df.sum()).to_numpy()
pd.DataFrame(J_fwd_par).style.format("{:.3f}")

,0,1,2,3
0,1.000,-1.021,-0.001,-0.000
1,0.000,2.021,-0.699,-0.000
2,0.000,0.000,1.699,-1.102
3,0.000,0.000,0.000,2.103


### Par to Forward Jacobian

Lets consider the process in reverse, defining our curve from forward swaps.

In [7]:
s_cv = SolvedCurve(
    nodes=nodes,
    swaps=list(fwd_swaps.keys()),
    obj_rates=list(fwd_swaps.values()),
    interpolation="log_linear",
    algorithm="levenberg_marquardt"
)
print(s_cv.iterate())

tolerance reached (levenberg_marquardt) after 8 iterations, func: 7.184307150707883e-16


In [8]:
for swap in fwd_swaps.keys():
    print(swap.rate(s_cv).real)

1.2099999938339168
2.0687923624754725
2.0602226928812533
1.9797176107904892


In [9]:
risk = {}
for swap in fwd_swaps.keys():
    risk.update({swap.end: swap.risk(s_cv)[:, 0]})
    
df = pd.DataFrame(risk, index=["1y", "1y1y", "2y3y", "5y5y"])
df.style.format("{:.3f}")

fwd tolerance reached (gauss_newton) after 2 iterations, func: 3.312324043978704e-24
bck tolerance reached (gauss_newton) after 2 iterations, func: 3.016302362268902e-25
fwd tolerance reached (gauss_newton) after 2 iterations, func: 3.598822892663517e-26
bck tolerance reached (gauss_newton) after 2 iterations, func: 1.1609188681289972e-24
fwd tolerance reached (gauss_newton) after 2 iterations, func: 8.095961141147461e-26
bck tolerance reached (gauss_newton) after 2 iterations, func: 7.253161871531963e-26
fwd tolerance reached (gauss_newton) after 2 iterations, func: 7.992955628448227e-26
bck tolerance reached (gauss_newton) after 2 iterations, func: 1.6490466691875814e-25


,2023-01-01 00:00:00,2024-01-01 00:00:00,2027-01-01 00:00:00,2032-01-01 00:00:00
1y,98.804,-0.000,0.000,-0.000
1y1y,0.000,96.802,0.000,-0.000
2y3y,0.000,0.000,279.082,-0.000
5y5y,0.000,0.000,0.000,429.645


In [10]:
risk = {}
for swap, rate in swaps.items():
    risk.update({swap.end: swap.risk(s_cv)[:, 0]})

df = pd.DataFrame(risk, index=["1y", "1y1y", "2y3y", "5y5y"])
df.style.format("{:.3f}")

,2023-01-01 00:00:00,2024-01-01 00:00:00,2027-01-01 00:00:00,2032-01-01 00:00:00
1y,98.804,98.804,98.804,98.804
1y1y,0.000,96.390,96.148,96.105
2y3y,0.000,0.000,278.130,277.746
5y5y,0.000,0.000,0.000,429.024


What we have done above is to risk par swaps against our forward curve. When we scale each of these columns to one we have effectively built a Jacobian transformation for par swaps to forward swaps.

In [11]:
J_par_fwd = (df / df.sum()).to_numpy()
pd.DataFrame(J_par_fwd).style.format("{:.3f}")

,0,1,2,3
0,1.000,0.506,0.209,0.110
1,0.000,0.494,0.203,0.107
2,0.000,0.000,0.588,0.308
3,0.000,0.000,0.000,0.476


### Testing the lossless nature of our numerical library

We should be able to convert and then convert back to obtain the same risks.

Suppose we convert 1000 10Y par risk to fwd risk and then back to par risk, what happens?

In [12]:
par_risk = np.array([0, 0, 0, 1000])[:, np.newaxis]
fwd_risk = np.matmul(J_par_fwd, par_risk)
fwd_risk

array([[109.57822051],
       [106.58425054],
       [308.03173736],
       [475.80579158]])

In [13]:
par_risk_reversed = np.matmul(J_fwd_par, fwd_risk)
par_risk_reversed

array([[ 4.65625473e-01],
       [ 6.24084380e-03],
       [-9.67230622e-01],
       [ 1.00049536e+03]])

This is close, but we have lost some precision due to either machine precision or numerical truncation.

The same can be seen if we compare the analytical inverses of the Jacobians.

In [14]:
pd.DataFrame(J_par_fwd - np.linalg.inv(J_fwd_par)).style.format("{:.4f}")

,0,1,2,3
0,0.0000,0.0011,0.0007,0.0003
1,0.0000,-0.0011,-0.0002,-0.0001
2,0.0000,0.0000,-0.0005,-0.0004
3,0.0000,0.0000,0.0000,0.0002


In [15]:
pd.DataFrame(J_fwd_par - np.linalg.inv(J_par_fwd)).style.format("{:.4f}")

,0,1,2,3
0,0.0000,0.0044,0.0000,0.0000
1,0.0000,-0.0044,0.0015,0.0000
2,0.0000,0.0000,-0.0015,-0.0010
3,0.0000,0.0000,0.0000,0.0010
